<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-snow-dots-dotsPositions.csv-file" data-toc-modified-id="Load-snow-dots-dotsPositions.csv-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load snow-dots dotsPositions.csv file</a></span></li><li><span><a href="#Write-data-to-file-with-dotsDB" data-toc-modified-id="Write-data-to-file-with-dotsDB-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Write data to file with dotsDB</a></span></li></ul></div>

The purpose of this notebook is to write the dots positions dumped by snow-dots to a dotsDB hdf5 file.

# Load snow-dots dotsPositions.csv file

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("../../data/pilot17_dotsPositions.csv")

In [4]:
# add a trialCount column if it doesn't exist
if "trialCount" not in data.columns:
    unique_dump_times = data['seqDumpTime'].unique()   
    tot_trials = unique_dump_times.size
    assert np.sum(np.diff(unique_dump_times)>=0) == tot_trials - 1 # confirm times are increasing
    
    # build the trialCount col outside of the dataframe
    trialCount = []
    for idx in range(tot_trials):
        # the long expression after the * symbol below simply counts how many rows data contains with a specific seqDumpTime value
        # the * symbol multiplies a list, it's a way to concatenate lists in Python
        trialCount += [idx + 1] * data[data["seqDumpTime"] == unique_dump_times[idx]].shape[0]
        
    data["trialCount"] = trialCount

In [5]:
data.head()

,xpos,ypos,isActive,isCoherent,frameIdx,seqDumpTime,pilotID,taskID,trialCount
0,0.337567,0.411096,1,1,1,75063.712102,17,1,1
1,0.362756,0.009781,0,0,1,75063.712102,17,1,1
2,0.646491,0.330317,0,0,1,75063.712102,17,1,1
3,0.813617,0.480657,1,1,1,75063.712102,17,1,1
4,0.210647,0.839180,0,0,1,75063.712102,17,1,1


In [6]:
def snowDots2DotsDBNormalizedFrames(df):
    """
    Casts a pandas.DataFrame corresponding to the .csv file outputted by some snow-dots programs into a list of 
    lists of frames, as dotsDB accepts them. The main mapping is the swapping of x and y.
    
    In dotsDB, x represents vertical position, y horizontal, and (0,0) is top left corner.
    In snow-dots, x represents horizontal position, y vertical, and (0,0) is top left corner.
      
    :param df: dataframe with columns "xpos", "ypos", "isActive", "isCoherent", "frameIdx", 
         "seqDumpTime", "pilotID", "taskID" 
    :type df: pandas.DataFrame
    """
    list_of_trials = []
    num_trials = np.max(df["trialCount"])
    for tr in range(num_trials):
        list_of_frames = []
        trial_data = df[df["trialCount"] == (tr+1)]
        
        num_frames = np.max(trial_data["frameIdx"])
        assert not np.isnan(num_frames), f'trial {tr+1}, num_frames is {num_frames}'
        
        for fr in range(num_frames):
            frame_data = trial_data[(trial_data["frameIdx"] == (fr+1)) & (trial_data["isActive"] == 1)]
            list_of_frames.append(np.array(frame_data[['ypos','xpos']]))  # here I swap xpos with ypos
        
        list_of_trials.append(list_of_frames)
    return list_of_trials

# Write data to file with dotsDB

In [7]:
import sys
import pprint
# pprint.pprint(sys.path)

In [8]:
sys.path.append('../modules/dots_db/dotsDB')

In [9]:
import dotsDB as dDB

In [10]:
db_filename = '../../data/pilot17_test.h5'

In [11]:
ddb_data = snowDots2DotsDBNormalizedFrames(data)

In [12]:
type(ddb_data)

list

In [13]:
len(ddb_data)

450

In [14]:
type(ddb_data[0])

list

In [15]:
len(ddb_data[0])

25

In [16]:
type(ddb_data[0][0])

numpy.ndarray

In [17]:
ddb_data[0][0].shape

(16, 2)

In [18]:
ddb_data[0][1].shape

(15, 2)

In [19]:
data.head()

,xpos,ypos,isActive,isCoherent,frameIdx,seqDumpTime,pilotID,taskID,trialCount
0,0.337567,0.411096,1,1,1,75063.712102,17,1,1
1,0.362756,0.009781,0,0,1,75063.712102,17,1,1
2,0.646491,0.330317,0,0,1,75063.712102,17,1,1
3,0.813617,0.480657,1,1,1,75063.712102,17,1,1
4,0.210647,0.839180,0,0,1,75063.712102,17,1,1


In [20]:
ddb_data[0][0][:2,]

array([[0.41109633, 0.33756732],
       [0.48065728, 0.81361659]])

In [21]:
parameters = dict(speed=2.1,density=90,coh_mean=50,coh_stdev=10,direction='right',num_frames=6,diameter=10)

In [22]:
n_trials=len(ddb_data)

In [23]:
stimulus = dDB.DotsStimulus(**parameters)

In [24]:
%%time
dDB.write_stimulus_to_file(stimulus, n_trials, db_filename, create_file=True, pre_generated_stimulus=ddb_data)

OSError: Can't write data (file write failed: time = Thu May  2 22:14:20 2019
, filename = '../../data/pilot17_test.h5', file descriptor = 44, errno = 28, error message = 'No space left on device', buf = 0x561f03a97b88, total write size = 3348144, bytes this sub-write = 3348144, bytes actually written = 18446744073709551615, offset = 129832157184)

In [25]:
dDB.inspect_db(db_filename)

{'intlv3_lftTrue_fr60_fs1.1_sp2.1_ds90_c50_cs10_dright_nf6_dm10_sc5.0_ppd55.4612_dts6_fw610': {'type': 'group',
  'attrs': [('coh_mean', 50),
   ('coh_stdev', 10),
   ('density', 90),
   ('diameter', 10),
   ('direction', 'right'),
   ('dot_size_in_pxs', 6),
   ('field_scale', 1.1),
   ('frame_rate', 60),
   ('frame_width_in_pxs', 610),
   ('interleaves', 3),
   ('limit_life_time', True),
   ('num_frames', 6),
   ('pixels_per_degree', 55.4612),
   ('speed', 2.1),
   ('stencil_radius_in_vis_angle', 5.0)]},
 'intlv3_lftTrue_fr60_fs1.1_sp2.1_ds90_c50_cs10_dright_nf6_dm10_sc5.0_ppd55.4612_dts6_fw610/px': {'type': 'dataset',
  'attrs': [],
  'shape': (100, 13395600)}}

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.
